In [6]:
#Imports

#Data
import pandas as pd
import numpy as np

#Date
import datetime as dt
from datetime import datetime

#Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#ML
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import tensorflow as tf
from sklearn.neural_network import MLPRegressor
from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.metrics import confusion_matrix,classification_report

from sklearn.preprocessing import OneHotEncoder

C:\Users\mmoy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [10]:
#Function that wrangles the data
def wrangle_data(file='data/xyz_corp_lending_data.csv'):
    
    #Read CSV file
    df=pd.read_csv(file,sep='\t')

    #Core - Drop columns (to be explained why for each column)
    df=df.drop(['policy_code','funded_amnt','funded_amnt_inv','out_prncp_inv','total_pymnt_inv','id','member_id','total_rec_prncp','policy_code','installment','pymnt_plan','application_type','next_pymnt_d','title','emp_title','sub_grade','grade'],axis=1)

    #Core - Creating a metric to get % of null values and selecting the columns that have less than 50%
    nullity=(df.isnull().sum()/len(df)*100).sort_values(ascending=False)
    df=df[nullity[nullity<50].index]

    #Core - Removing text from emp_length and term columns and converting to float
    df['emp_length']=df['emp_length'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
    df['term']=df['term'].str.replace('[A-Za-z\s+<>]+', '').astype('float')

    #Core - Converting Date Columns to YYYYMMDD format and float
    date_cols=['last_pymnt_d','last_credit_pull_d','issue_d','earliest_cr_line']
    for col in date_cols:
        df[col]=pd.to_datetime(df[col]).dt.strftime('%Y%m%d').astype(float)

    #Filling na with median or mode
    med=['tot_cur_bal','tot_coll_amt','total_rev_hi_lim','revol_util']
    mod=['emp_length','last_pymnt_d','collections_12_mths_ex_med','last_credit_pull_d']
    for col in med:
        df[col]=df[col].fillna(df[col].median())

    for col in mod:
        df[col]=df[col].fillna(df[col].mode()[0])

    #Cleaning up zip_code to only have numbers and concatenating with addr_state. Maybe remove add_state as zip_code already covers it
    df['zip_code']=df['zip_code'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
    #df['address']=df['zip_code']+df['addr_state']

    #Encoding categorical values 
    cat_columns=['purpose','verification_status','home_ownership','initial_list_status','addr_state']
    df[cat_columns] = df[cat_columns].astype('category').apply(lambda x: x.cat.codes)

    return df
wrangle_data().to_csv('data/wrang_xyz_data.csv',index=False)

C:\Users\mmoy\AppData\Local\Temp\ipykernel_22008\2300198398.py:5: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(file,sep='\t')
C:\Users\mmoy\AppData\Local\Temp\ipykernel_22008\2300198398.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['emp_length']=df['emp_length'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
C:\Users\mmoy\AppData\Local\Temp\ipykernel_22008\2300198398.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df['term']=df['term'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
C:\Users\mmoy\AppData\Local\Temp\ipykernel_22008\2300198398.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['zip_code']=df['zip_code'].str.replace('[A-Za-z\s+<>]+', '').astype('float')
